In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import torch
import torch.nn.functional as F

from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle
from string import punctuation
from german_parser.model import TigerModel
from german_parser.util import BatchUnionFind

from german_parser.util.c_and_d import Dependency, DependencyTree, ConstituentTree, Terminal
from collections import defaultdict

# Define Methods

In [9]:
def find_tree(model: TigerModel, input: tuple[torch.Tensor, torch.Tensor], new_words_dict: dict[int, str] | None):
    """NOTE: indices are 1-indexed. index 0 corresponds to virtual root of D-tree (which is different from virtual root of C-tree)

    Args:
        model (TigerModel): _description_
        input (tuple[torch.Tensor, torch.Tensor]): _description_
        new_words_dict (dict[int, str] | None): _description_
    """    
    x, lengths = input
    self_attention, constituent_labels, attachment_orders, indices = model.forward(input, new_words_dict)
    # self_attention has size (B, T, T + 1)

    B, T, *_ = self_attention.shape
    uf = BatchUnionFind(B, model.beam_size, N=T + 1, device=model.dummy_param.device)

    # initialise beams by finding top-K most probable root nodes
    best_roots = self_attention[:, :, 0].topk(k=model.beam_size, dim=-1)

    current_root_indices = best_roots.indices # (B, K)
    joint_logits = best_roots.values # (B, K)

    edges = torch.zeros(B, model.beam_size, T, dtype=torch.long, device=model.dummy_param.device) # (B, K, T); m[b, k, t - 1] is the 1-indexed parent of 1-indexed node t, in batch b, beam k

    same_as_beams = torch.zeros(B, model.beam_size) # m[b, k] == 1 if beam k in batch b is equal to beam 1 in batch b. allows us to keep track of duplicates. duplicates are when m[b, k] != k
    
    def beams_are_unique():
        return same_as_beams == torch.arange(model.beam_size).repeat(B, 1) # return where m[b, k] == same_as_beams[b, k] != k


    for t in range(T):
        relevant_batches = t < lengths # used for updating the final arcs. otherwise, we get nans in batch b if t >= sentence_length[b]

        arc_probs = self_attention[:, t, :].log_softmax(dim=-1) # (B, T + 1)

        candidate_joint_logits = joint_logits[:, :, None] + arc_probs[:, None, :] # (B, K, T + 1); the heuristic we would like to maximise

        children = torch.tensor(t + 1, device=model.dummy_param.device) # all batches and beams share the same child index (1-indexed)
        parents = torch.arange(0, T + 1, 1, device=model.dummy_param.device, dtype=torch.long).repeat(B, model.beam_size, 1) # (B, K, T + 1), where m[b, k, t] = t to represent the index of each parent 

        # prevent cycles
        disable_mask = uf.is_same_set(children.expand_as(parents), parents) # (B, K, T + 1); m[b, k, s + 1] is true if in batch b, beam k, joining child (t + 1) and parent (s + 1) would lead to a cycle
        # avoid setting words as head that are beyond the end of the sentence
        disable_mask[:, :, 1:] |= ~indices.unsqueeze(1).to(device=model.dummy_param.device) 
        # force beams to be unique
        disable_mask |= ~beams_are_unique()[:, :, None]
        # force root indices to be enabled
        disable_mask[:, :, 1:][current_root_indices == t] = True # for the batches and beams where t would be a root node, t's parent must be 0 (cannot be 1:)
        # prevent other indices from becoming root
        disable_mask[:, :, 0][current_root_indices != t] = True
        candidate_joint_logits[disable_mask] = -torch.inf # these indices can never be a maximiser

        flattened_top_candidate_idx = candidate_joint_logits.flatten(-2, -1).topk(k=model.beam_size, dim=-1).indices # (B, K) in range [0, (K * T + 1)); for each batch, find top 10 best performing parent-beam combinations
        
        top_parents = parents.flatten(-2, -1).gather(index=flattened_top_candidate_idx, dim=-1) # (B, K); for each batch and beam, get the 1-indexed id of the parent we want to attach

        batch_names = torch.arange(model.beam_size, device=model.dummy_param.device, dtype=torch.long).view(1, -1, 1).expand(B, -1, T + 1).flatten(-2, -1) # (B, K, T + 1); m[b, k, s] = k for all b, s, k
        used_batches = batch_names.gather(index=flattened_top_candidate_idx, dim=-1) # (B, K) where each element is in the range [0, K). m[b, k] tells you what the kth new beam should be

        same_as_beams = used_batches # we have copied over these beams, so for now, these beams must be equal

        new_data = uf.data.gather(index=used_batches.unsqueeze(-1).expand_as(uf.data), dim=1)
        new_rank = uf.rank.gather(index=used_batches.unsqueeze(-1).expand_as(uf.rank), dim=1)
        new_edges = edges.gather(index=used_batches.unsqueeze(-1).expand_as(edges), dim=1)
        new_joint_logits = candidate_joint_logits.flatten(-2, -1).gather(index=flattened_top_candidate_idx, dim=-1)
        new_roots = current_root_indices.gather(index=used_batches, dim=-1)
        

        uf.data[relevant_batches] = new_data[relevant_batches]
        uf.rank[relevant_batches] = new_rank[relevant_batches]
        edges[relevant_batches] = new_edges[relevant_batches]
        joint_logits[relevant_batches] = new_joint_logits[relevant_batches]
        current_root_indices[relevant_batches] = new_roots[relevant_batches]

        uf.union(children.expand_as(top_parents), top_parents)

        edges[:, :, t] = top_parents
        new_unique_beams = top_parents.gather(index=same_as_beams, dim=-1) != top_parents  # indicates the beams that have BECOME unqiue: given any batch b, suppose beam j was a copy of beam k. suppose the new parent for beam j is different to the new parent of beam k. then m[b, j] = True
        same_as_beams[new_unique_beams] = torch.arange(model.beam_size).repeat(B, 1)[new_unique_beams]

        pass

    num_labels = constituent_labels.shape[-1]
    num_attachment_orders = attachment_orders.shape[-1]

    best_edges = edges[torch.arange(edges.shape[0]), joint_logits.argmax(dim=-1)] # (B, T) containing elements in range [0, T + 1), where m[b, t - 1] denotes the 1-indexed parent of 1-indexed node t

    label_logits_best_edges = constituent_labels.gather(index=best_edges[:, :, None, None].expand(-1, -1, -1, num_labels), dim=2).squeeze(2)
    attachment_logits_best_edges = attachment_orders.gather(index=best_edges[:, :, None, None].expand(-1, -1, -1, num_attachment_orders), dim=2).squeeze(2)

    labels_best_edges = label_logits_best_edges.argmax(-1)
    attachment_orders_best_edges = attachment_logits_best_edges.argmax(-1)

    return best_edges, labels_best_edges, attachment_orders_best_edges, (edges, joint_logits)

In [10]:
def get_bracket(c_tree: ConstituentTree, node: int|None=None, ignore_pre_terminal_sym: bool=False, ignore_non_terminal_sym: bool=False, ignore_all_syms: bool=False):
    if node is None:
        return get_bracket(c_tree,
                           node=c_tree.root,
                           ignore_pre_terminal_sym=ignore_pre_terminal_sym,
                           ignore_non_terminal_sym=ignore_non_terminal_sym,
                           ignore_all_syms=ignore_all_syms
                           )
    
    c = c_tree.constituents[node]
    if c.is_pre_terminal:
        return f"({'?' if ignore_pre_terminal_sym or ignore_all_syms else c.sym} {c.id}={c_tree.terminals[node].word})"
    
    res = f"({'?' if ignore_non_terminal_sym or ignore_all_syms else c.sym} "
    for child in c.children:
        res += get_bracket(c_tree,
                           node=child,
                           ignore_pre_terminal_sym=ignore_pre_terminal_sym,
                           ignore_non_terminal_sym=ignore_non_terminal_sym,
                           ignore_all_syms=ignore_all_syms
                           )
        
    res += ")"
    return res

In [11]:
def create_d_tree(heads: torch.Tensor, labels: torch.Tensor, orders: torch.Tensor, word_ids: torch.Tensor, word_dict: dict[int, str]):
    num_words = len(heads)
    modifiers: dict[int, dict[int, list[Dependency]]] = defaultdict(lambda: defaultdict(lambda x=None: []))
    terminals: dict[int, Terminal] = {}

    for child, (head, label, order, word) in enumerate(zip(heads, labels, orders, word_ids), start=1):
        head = int(head.item())
        label = int(label.item())
        order = int(order.item())
        word = int(word.item())

        terminals[child] = Terminal(
            idx=child,
            word=word_dict[word]
        )

        # child with head of 0 is the root node of the d-tree; don't add it to modifiers dict
        if head == 0:
            continue
        
        modifiers[head][order].append(Dependency(head=head, modifier=child, sym=inverse_sym_dict[label]))

    # with an ordering, all arcs must have the same symbol between dependency and head
    for h in modifiers:
        for o in modifiers[h]:
            dependencies = modifiers[h][o]
            closest_dependency_to_head = max(dependencies, key=lambda d: abs(d.head - d.modifier))
            for d in dependencies:
                if d.sym != closest_dependency_to_head.sym:
                    print(f"Warning: arc symbol from {d.modifier} to {d.head} ({d.sym}) does not match arc from closest dependency {closest_dependency_to_head.modifier} ({closest_dependency_to_head.sym}). Setting this to ({closest_dependency_to_head.sym})")
                    d.sym = closest_dependency_to_head.sym
            pass

    return DependencyTree(modifiers=modifiers, terminals=terminals, num_words=num_words)

def create_c_tree(heads: torch.Tensor, labels: torch.Tensor, orders: torch.Tensor, word_ids: torch.Tensor, word_dict: dict[int, str]):
    d_tree = create_d_tree(heads, labels, orders, word_ids, word_dict)
    return ConstituentTree.from_d_tree(d_tree)

# Playing Around

In [6]:
(train_dataloader, train_new_words), (dev_dataloader, dev_new_words), (test_dataloader, test_new_words), character_set, character_flag_generators, inverse_word_dict, inverse_sym_dict = pickle.load(open("required_vars.pkl", "rb"))
model: TigerModel = pickle.load(open("./models/epoch_25_cpu_eval.pickle", "rb"))
dl_en = enumerate(test_dataloader)
_, (x, l, target_ex, target_lab, target_att) = next(dl_en)

In [12]:
best_edges, labels_best_edges, attachment_orders_best_edges, (edges, joint_logits) = find_tree(model, (x, l), test_new_words)

In [13]:
joint_logits[0]

tensor([ 1.1600,  0.6360,  0.4139,  0.3725,  0.3133, -0.8975, -1.4497, -1.9736,
        -2.1114, -2.1957], grad_fn=<SelectBackward0>)

In [15]:
s_num = 3

words = x[s_num, :l[s_num]].to("cpu")

ex = best_edges[s_num, :l[s_num]].to("cpu")
lab = labels_best_edges[s_num, :l[s_num]].to("cpu")
att = attachment_orders_best_edges[s_num, :l[s_num]].to("cpu")

t_ex = target_ex[s_num, :l[s_num]].to("cpu")
t_lab = target_lab[s_num, :l[s_num]].to("cpu")
t_att = target_att[s_num, :l[s_num]].to("cpu")

In [16]:
edges = []

for i, (i_parent, i_lab, i_att) in enumerate(zip(ex, lab, att), 1):
    edges.append(f"{{{i_parent}->{i}, \"{inverse_sym_dict[i_lab.item()]}#{i_att}\"}}")

print(f"TreePlot[{{{', '.join(edges)}}}, Top, 0, VertexLabels -> Automatic, DirectedEdges -> True]")

TreePlot[{{14->1, "VROOT#2"}, {3->2, "NP#1"}, {14->3, "S#1"}, {14->4, "VROOT#2"}, {6->5, "NP#1"}, {3->6, "NP#1"}, {14->7, "VROOT#2"}, {12->8, "S#1"}, {12->9, "VP#1"}, {9->10, "CNP#1"}, {9->11, "CNP#1"}, {6->12, "NP#1"}, {14->13, "VROOT#2"}, {0->14, "DROOT#1"}, {20->15, "VP#1"}, {20->16, "VP#1"}, {16->17, "PP#1"}, {17->18, "CNP#1"}, {17->19, "CNP#1"}, {14->20, "S#1"}, {14->21, "VROOT#2"}, {24->22, "VP#1"}, {24->23, "VP#1"}, {20->24, "VP#1"}, {14->25, "VROOT#2"}}, Top, 0, VertexLabels -> Automatic, DirectedEdges -> True]


In [17]:
target_edges = []

for i, (i_parent, i_lab, i_att) in enumerate(zip(t_ex, t_lab, t_att), 1):
    target_edges.append(f"{{{i_parent}->{i}, \"{inverse_sym_dict[i_lab.item()]}#{i_att}\"}}")

print(f"TreePlot[{{{', '.join(target_edges)}}}, Top, 0, VertexLabels -> Automatic, DirectedEdges -> True]")

TreePlot[{{14->1, "VROOT#2"}, {3->2, "NP#1"}, {14->3, "S#1"}, {14->4, "VROOT#2"}, {6->5, "NP#1"}, {3->6, "NP#1"}, {14->7, "VROOT#2"}, {12->8, "S#1"}, {12->9, "S#1"}, {9->10, "CNP#1"}, {9->11, "CNP#1"}, {6->12, "NP#1"}, {14->13, "VROOT#2"}, {0->14, "DROOT#1"}, {20->15, "VP#1"}, {20->16, "VP#1"}, {16->17, "PP#1"}, {17->18, "CNP#1"}, {17->19, "CNP#1"}, {14->20, "S#1"}, {14->21, "VROOT#2"}, {24->22, "VP#1"}, {24->23, "VP#1"}, {20->24, "VP#1"}, {14->25, "VROOT#2"}}, Top, 0, VertexLabels -> Automatic, DirectedEdges -> True]


In [18]:
c_tree = create_c_tree(heads=ex, labels=lab, orders=att, word_ids=words, word_dict={**inverse_word_dict, **{-i: w for i, w in test_new_words.items()}})
t_c_tree = create_c_tree(heads=t_ex, labels=t_lab, orders=t_att, word_ids=words, word_dict={**inverse_word_dict, **{-i: w for i, w in test_new_words.items()}})

In [19]:
get_bracket(c_tree)

"(VROOT (S (? 14=werde)(NP (? 3=Großteil)(? 2=Ein)(NP (? 6=Entscheidungen)(? 5=der)(S (? 12=betreffen)(? 8=die)(CNP (? 9=Einwanderer)(? 10=und)(? 11=Minderheiten)))))(VP (? 20=getroffen)(? 15=bislang)(PP (? 16=auf)(CNP (? 17=Bundes-)(? 18=und)(? 19=Landesebene)))(VP (? 24=anzuhören)(? 22=ohne)(? 23=diese))))(? 1=``)(? 4='')(? 7=,)(? 13=,)(? 21=,)(? 25=.))"

In [20]:
get_bracket(t_c_tree)

"(VROOT (S (? 14=werde)(NP (? 3=Großteil)(? 2=Ein)(NP (? 6=Entscheidungen)(? 5=der)(S (? 12=betreffen)(? 8=die)(CNP (? 9=Einwanderer)(? 10=und)(? 11=Minderheiten)))))(VP (? 20=getroffen)(? 15=bislang)(PP (? 16=auf)(CNP (? 17=Bundes-)(? 18=und)(? 19=Landesebene)))(VP (? 24=anzuhören)(? 22=ohne)(? 23=diese))))(? 1=``)(? 4='')(? 7=,)(? 13=,)(? 21=,)(? 25=.))"

In [21]:
c_tree.draw()

(<toyplot.canvas.Canvas at 0x7f299e7d9c50>,
 <toytree.Render.ToytreeMark at 0x7f299e7803d0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td8906defb804441eb57ebc7f8befd580"> 19.? 18.? 17.? 16.? 23.? 22.? 24.? 15.? 20.? 11.? 10.? 9.? 8.? 12.? 5.? 6.? 2.? 3.? 14.? 25.? 21.? 13.? 7.? 4.? 1.? CNP CNP S PP VP NP VP NP S VROOT Landesebene und Bundes- auf diese ohne anzuhören bislang getroffen Minderheiten und Einwanderer die betreffen der Entscheidungen Ein Großteil werde . , , , '' ``

In [22]:
t_c_tree.draw()

(<toyplot.canvas.Canvas at 0x7f29adfd2210>,
 <toytree.Render.ToytreeMark at 0x7f29b00c4410>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9deeb3a647d0411eb879b11f1625175c"> 19.? 18.? 17.? 16.? 23.? 22.? 24.? 15.? 20.? 11.? 10.? 9.? 8.? 12.? 5.? 6.? 2.? 3.? 14.? 25.? 21.? 13.? 7.? 4.? 1.? CNP CNP S PP VP NP VP NP S VROOT Landesebene und Bundes- auf diese ohne anzuhören bislang getroffen Minderheiten und Einwanderer die betreffen der Entscheidungen Ein Großteil werde . , , , '' ``